In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler, PowerTransformer, QuantileTransformer

class ScalingReverser:
    """
    معکوس‌سازی عملیات Scaling/Normalization روی یک ستون.

    Parameters
    ----------
    method : str
        نام متد scaling که باید معکوس شود.
        پشتیبانی شده: 'StandardScaler', 'MinMaxScaler', 'RobustScaler',
        'MaxAbsScaler', 'PowerTransformer', 'QuantileTransformer'
    config : dict
        تنظیمات شامل پارامترهای fitted شده (مانند mean, scale, lambdas, ...)
    """
    def __init__(self, method, config):
        self.method = method
        self.config = config
        self.scaler_ = None

    def transform(self, dataset, column):
        """
        اعمال معکوس‌سازی روی ستون مشخص و افزودن نتیجه به دیتاست.

        Parameters
        ----------
        dataset : pd.DataFrame
            دیتاست ورودی.
        column : str
            نام ستونی که باید معکوس شود.

        Returns
        -------
        pd.DataFrame
            دیتاست با ستون جدید '{column}_reversed'.
        """
        if not isinstance(dataset, pd.DataFrame):
            raise TypeError("dataset must be a pandas DataFrame")
        if column not in dataset.columns:
            raise ValueError(f"Column '{column}' not found in dataset")

        X_col = dataset[[column]].values  # شکل (n_samples, 1)

        if self.method == 'StandardScaler':
            mean = self.config.get('mean')
            scale = self.config.get('scale')
            if mean is None or scale is None:
                raise ValueError("StandardScaler requires 'mean' and 'scale' in config")
            self.scaler_ = StandardScaler()
            self.scaler_.mean_ = np.array(mean)
            self.scaler_.scale_ = np.array(scale)
            reversed_col = self.scaler_.inverse_transform(X_col).flatten()

        elif self.method == 'MinMaxScaler':
            min_ = self.config.get('min')
            scale_ = self.config.get('scale')
            if min_ is None or scale_ is None:
                raise ValueError("MinMaxScaler requires 'min' and 'scale' in config")
            self.scaler_ = MinMaxScaler()
            self.scaler_.min_ = np.array(min_)
            self.scaler_.scale_ = np.array(scale_)
            reversed_col = self.scaler_.inverse_transform(X_col).flatten()

        elif self.method == 'RobustScaler':
            center_ = self.config.get('center')
            scale_ = self.config.get('scale')
            if center_ is None or scale_ is None:
                raise ValueError("RobustScaler requires 'center' and 'scale' in config")
            self.scaler_ = RobustScaler()
            self.scaler_.center_ = np.array(center_)
            self.scaler_.scale_ = np.array(scale_)
            reversed_col = self.scaler_.inverse_transform(X_col).flatten()

        elif self.method == 'MaxAbsScaler':
            scale_ = self.config.get('scale')
            if scale_ is None:
                raise ValueError("MaxAbsScaler requires 'scale' in config")
            self.scaler_ = MaxAbsScaler()
            self.scaler_.scale_ = np.array(scale_)
            reversed_col = self.scaler_.inverse_transform(X_col).flatten()

        elif self.method == 'PowerTransformer':
            method_pt = self.config.get('method', 'yeo-johnson')
            lambdas_ = self.config.get('lambdas')
            if lambdas_ is None:
                raise ValueError("PowerTransformer requires 'lambdas' in config")
            self.scaler_ = PowerTransformer(method=method_pt)
            self.scaler_.lambdas_ = np.array(lambdas_)
            reversed_col = self.scaler_.inverse_transform(X_col).flatten()

        elif self.method == 'QuantileTransformer':
            n_quantiles_ = self.config.get('n_quantiles')
            quantiles_ = self.config.get('quantiles')
            references_ = self.config.get('references')
            if n_quantiles_ is None or quantiles_ is None or references_ is None:
                raise ValueError("QuantileTransformer requires 'n_quantiles', 'quantiles', and 'references' in config")
            self.scaler_ = QuantileTransformer(n_quantiles=n_quantiles_)
            self.scaler_.quantiles_ = np.array(quantiles_)
            self.scaler_.references_ = np.array(references_)
            reversed_col = self.scaler_.inverse_transform(X_col).flatten()

        elif any(x in self.method for x in ['Normalizer', 'Clipping']):
            raise NotImplementedError(f"Inverse transform not supported for {self.method}")

        else:
            raise ValueError(f"Unknown scaling method: {self.method}")

        dataset = dataset.copy()
        dataset[f"{column}_reversed"] = reversed_col
        return dataset